<a href="https://colab.research.google.com/github/lucascviana/Meus-Projetos/blob/main/Predi%C3%A7%C3%A3o_de_Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Predição de Diabetes
Workflow:

Problema: Prever a partir de uma série de variáveis se a pessoa tem diabates ou não

Passos:
Importar dados e bibliotecas
Processamento e tratamento dos dados (Normalização para deixar tudo na escala de 0 a 1)
Validação Cruzada
Treinamento do Modelo de Support Vector Machine
Validação e teste do Modelo


Modelo Support Vector Machine:
Conceito: Modelo de classificação e regressão mas com boa aplicação para classificação -

A ideia central por trás do SVM é encontrar um hiperplano de separação ótimo entre classes em um espaço de características de alta dimensão. Este hiperplano é escolhido de forma que maximize a margem, que é a distância entre as instâncias mais próximas de cada classe, chamadas vetores de suporte.

Detalhes:

Margem: Distância entre o hiperplano e os vetores de suporte - uma vez otimizados, facilita a separação dos grupos

Kernel Trick: Transformação dos dados e dimensões para facilitar a classificação e separação


In [ ]:
#importando as bibliotecas de uso

import pandas as pd #tabelas como objetos
import numpy as np # foco nas operações em arrays
from sklearn import svm #biblioteca do modelo
from sklearn.preprocessing import StandardScaler #função de tratamento para normalizar os dados em uma escala
from sklearn.model_selection import train_test_split #função de separação do dataset em treino e teste
from sklearn.metrics import accuracy_score #função que mensura os erros do dataset

In [ ]:
#load do dataset

df = pd.read_csv('/content/diabetes.csv')

#print(df) #forma 1 de ver o df
df.head() #forma mais bonita e organizada

#1 a pessoa possui diabetes
#0 a pessoa não possui diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
#entendimento do dataset
#quantas colunas e linhas possuem?

df.shape #768 linhas e 9 colunas/variáveis


#quais são as médias, desvio padrão e quartos do conjunto?
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
#distribuição de quem tem e quem não tem diabetes

df['Outcome'].value_counts()
#500 linhas não possuem
#268 linhas possuem

0    500
1    268
Name: Outcome, dtype: int64

In [ ]:
#entendimento por grupo de pessoas - diabéticos e não diabéticos
df.groupby('Outcome').mean()

#é possivel ver diferenças nas médias, indicando que as variaveis tem relação com o output

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [ ]:
#separação dos dados para não utilizar a coluna de output no treinamento
x = df.drop(columns='Outcome', axis= 1) #tiro a coluna de output do conjunto

y = df['Outcome'] #crio uma coluna somente com o outpt

print(x, y)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [ ]:
#padronização dos dados de features
normalizador = StandardScaler()

#interessante aqui a relação de instância, bibliotecas e atributos  -
#no caso há uma criação de uma variável/instância de uma função da biblioteca, assim a variável recebe os atributos/métodos dessa função
#sendo possível "chamar" tais atributos através do objeto em questão, nesse caso o normalizador com a função StandScaler

normalizador.fit(x) #fitar os dados do treinamento

StandardScaler()

In [ ]:
#criação de uma varíavel com o novo dataset

data_normalizada = normalizador.transform(x)

print(data_normalizada)


[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [ ]:
#reorganizando os dados para o modelo
x= data_normalizada
y = df['Outcome']

In [ ]:
#divisão do dataset em dados de treino e teste
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y, random_state=2)

In [ ]:
#quantificação dos dadaos de cada conjunto
print(x.shape, x_train.shape,x_test.shape)

(768, 8) (614, 8) (154, 8)


Treinamento do Modelo

In [ ]:
#criação da instância/objeto para uso dos atributos/métodos da função

classificador = svm.SVC(kernel='linear')

In [ ]:
#chamando o objeto para ter o metodo atrelado a ele - puxar sua função e inserir os dados e seus respectivos rótulos
classificador.fit(x_train, y_train)

SVC(kernel='linear')

In [ ]:
#previsão dos valores - chamada da instância para acesso ao método
x_train_previsao = classificador.predict(x_train)


#validação do modelo nos dados de treino
#acurácia nos dados de treino
acuracia_dados_treinamento =accuracy_score(x_train_previsao, y_train)

print(acuracia_dados_treinamento)

0.7866449511400652


In [ ]:
#previsão dos valores - chamada da instância para acesso ao método
x_train_previsao = classificador.predict(x_train)


#validação do modelo nos dados de treino
#acurácia nos dados de treino
acuracia_dados_treinamento =accuracy_score(x_train_previsao, y_train)

print(acuracia_dados_treinamento)

0.7866449511400652


In [ ]:
input_data = (4,110,92,0,0,37.6,0.191,30)

#convertendo o input para np array
input_data_as_np_array = np.asarray(input_data)

#mudando o formato do array para previsão de uma instância - foco no modelo entender para prever somente um valor a partir de um valor também
input_data_reshaped = input_data_as_np_array.reshape(1,-1) # só pega uma instância e preve

#também é preciso normalizar os dados de input
std_data = normalizador.transform(input_data_reshaped)

print(std_data)


prediction = classificador.predict(std_data) #o modelo está na variavel classiifer e puxei a função de prever
print (prediction)

if (prediction == 0):
  print('Pessoa não é diabetica')
else:
  print ('A pessoa é diabética')
##DEU CERTO PARA ESSES VALORES
input_normalizado = normalizador.transform(input_data_reshaped)

[[ 0.04601433 -0.34096773  1.18359575 -1.28821221 -0.69289057  0.71168975
  -0.84827977 -0.27575966]]
[0]
Pessoa não é diabetica


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
